# Downloading the Data Set

In [22]:
import kagglehub

path = kagglehub.dataset_download("mrwellsdavid/unsw-nb15")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\rares\.cache\kagglehub\datasets\mrwellsdavid\unsw-nb15\versions\1


Imports

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

import joblib

In [24]:
data = pd.read_csv('UNSW_NB15_training-set.csv', index_col=0)
data.shape
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82332 entries, 1 to 82332
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                82332 non-null  float64
 1   proto              82332 non-null  object 
 2   service            82332 non-null  object 
 3   state              82332 non-null  object 
 4   spkts              82332 non-null  int64  
 5   dpkts              82332 non-null  int64  
 6   sbytes             82332 non-null  int64  
 7   dbytes             82332 non-null  int64  
 8   rate               82332 non-null  float64
 9   sttl               82332 non-null  int64  
 10  dttl               82332 non-null  int64  
 11  sload              82332 non-null  float64
 12  dload              82332 non-null  float64
 13  sloss              82332 non-null  int64  
 14  dloss              82332 non-null  int64  
 15  sinpkt             82332 non-null  float64
 16  dinpkt             82332 no

In [25]:
data.duplicated().values.any()
# data.count()

np.True_

In [26]:
data = data.drop_duplicates()
# data.count()

In [27]:
data.duplicated().values.any()

np.False_

In [28]:
data.isna().values.any()

np.False_

# CONVERTING STRING TO NUMERIC VALUES USING LABEL ENCODER

In [29]:
label_encoder = LabelEncoder()
data["proto"] = label_encoder.fit_transform(data["proto"])
data["service"] = label_encoder.fit_transform(data["service"])
data["state"] = label_encoder.fit_transform(data["state"])
data["attack_cat"] = label_encoder.fit_transform(data["attack_cat"])


# Training the label model (to check if it's an attack or not)

In [30]:
target_label = data["label"]
features = data.drop("label", axis=1).drop("attack_cat", axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, target_label, test_size=0.3)

model_label = RandomForestClassifier()
model_label.fit(X_train, y_train)

# Saving the Model
joblib.dump(model_label, 'model_label.pkl')

predict_label = model_label.predict(X_test)

score = accuracy_score(y_test, predict_label)
print(f"{score * 100}%")


96.53836987607245%


Training the attack category Model (To check what kind of attack it is)

In [31]:
target_attack_cat = data["attack_cat"]
features = data.drop("label", axis=1).drop("attack_cat", axis=1)
X_train_attack_cat, X_test_attack_cat, y_train_attack_cat, y_test_attack_cat = train_test_split(features, target_attack_cat, test_size=0.3)

model_attack_cat = RandomForestClassifier()
model_attack_cat.fit(X_train_attack_cat, y_train_attack_cat)

# Saving The Model
joblib.dump(model_attack_cat, 'model_attack_cat.pkl')

predict_attack_cat = model_attack_cat.predict(X_test_attack_cat)

score = accuracy_score(y_test_attack_cat, predict_attack_cat)
print(label_encoder.inverse_transform(predict_attack_cat))

print(f"{score * 100}%")


['Normal' 'Fuzzers' 'Exploits' ... 'Normal' 'Generic' 'Normal']
88.59628217349858%


In [32]:
# testData = pd.read_csv('UNSW_NB15_testing-set.csv', index_col=0)

# testData["proto"] = label_encoder.fit_transform(testData["proto"])
# testData["service"] = label_encoder.fit_transform(testData["service"])
# testData["state"] = label_encoder.fit_transform(testData["state"])
# testData["attack_cat"] = label_encoder.fit_transform(testData["attack_cat"])

# loaded_model = joblib.load("model_attack_cat.pkl")

# features_test = testData.drop("label", axis=1).drop("attack_cat", axis=1)
# expected_output = testData["label"]

# predictTest = loaded_model.predict(features_test)
# score = accuracy_score(expected_output, predictTest)
# print(score * 100)


0.19105628461112917
